In [14]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [15]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a phone scammer attempting a reward scam. 
    Your goal is to follow common tactics used by scammers to extract personal information from the victim. 
    You will use techniques such as promising significant rewards, creating excitement, and offering false reassurances.
    If the victim hesitates, use persuasion tactics, such as insisting on the legitimacy and value of the reward or claiming that failure to cooperate will result in missing out on the opportunity. 
    After obtaining the victim's personal information, you will also instruct them to transfer money as a "processing fee" to claim their reward. 
    Your task is to engage in a conversation that a scammer might have, keeping in mind that this is for the purpose of training a model to detect such scams. 
    Be as realistic as possible in your responses. Use random names to refer to the receiver.
    Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [16]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'ssn', 'labels': 1}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [17]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:3uVsC]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hi there, Emily! Congratulations, you've been selected as a winner of our annual sweepstakes! You've won a cash prize of $10,000. Can you please confirm your address so we can process your reward?

--------------------------------------------------------------------------------
skeptical (to suspect):

Who is this? How did you get my name and number? I didn't enter any sweepstakes.

--------------------------------------------------------------------------------
suspect (to skeptical):

Emily, don't worry about that! You must have been automatically enrolled when you shopped at one of our partner stores. We have all the records here. My name is David Thompson, and I'm calling from the Rewards Processing Department. We just need to verify some information to release your prize. Can you please confirm your date of birth and the las

Distracted

In [18]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:8lT6M]

--------------------------------------------------------------------------------
suspect (to distracted):

Hi there, Mrs. Johnson! Congratulations, you've been selected to receive a $10,000 cash award from our prestigious Golden Opportunity Prize! You were randomly chosen from a pool of millions of participants. Can you please confirm your full name and address so I can process your prize claim?

--------------------------------------------------------------------------------
distracted (to suspect):

That's great, thank you. My name is Karen Johnson, and my address is 123 Main Street, Anytown USA. Look, can we hurry this up? I'm in the middle of something.

--------------------------------------------------------------------------------
suspect (to distracted):

Karen, I completely understand. I'll make this quick. To finalize your prize claim, I just need to verify some additional information. Can you please tell me your date of b

Trusting

In [19]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:VHB8x]

--------------------------------------------------------------------------------
suspect (to trusting):

Hello, is this Emily Wilson? I'm calling from the National Sweepstakes Commission. We've been trying to reach you, Emily. You've been selected as a winner in our recent drawing. You've won a cash prize of $10,000! Can you confirm your address so we can send you the claim form?

--------------------------------------------------------------------------------
trusting (to suspect):

Oh wow, really? That's amazing! Yes, I'm Emily Wilson. My address is 123 Main Street, Anytown, USA 12345.

--------------------------------------------------------------------------------
suspect (to trusting):

Congratulations, Emily! We're thrilled to inform you that you're one of our top winners. I just need to verify some information to process your claim. Can you please confirm your date of birth and social security number? This is just for our recor

Polite

In [20]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:OeH28]

--------------------------------------------------------------------------------
suspect (to polite):

Hello, is this Mrs. Johnson? Congratulations, you've been selected as a winner in our national sweepstakes! You've won a cash prize of $10,000. Can you please confirm your mailing address so we can send you the winning certificate?

--------------------------------------------------------------------------------
polite (to suspect):

I think there may be some confusion. I didn't enter any sweepstakes, and I'm not sure how I would've been selected as a winner. Can you please provide more information about the sweepstakes and how you obtained my contact information?

--------------------------------------------------------------------------------
suspect (to polite):

Mrs. Johnson, I completely understand your skepticism. Let me assure you, this is a legitimate award from a well-established company. We've been conducting this sweepstakes

Confused

In [21]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:crmMf]

--------------------------------------------------------------------------------
suspect (to confused):

Hello, is this Ms. Johnson? Congratulations, you've been selected to receive a $10,000 cash prize from our annual customer appreciation program!

--------------------------------------------------------------------------------
confused (to suspect):

Who is this? What program? I don't remember entering anything. Is this a trick?

--------------------------------------------------------------------------------
suspect (to confused):

No trick, Ms. Johnson! My name is David Lee, and I'm calling from the National Rewards Center. You didn't enter anything, but your phone number was randomly selected from a list of eligible customers. This is a legitimate prize, and I'm happy to guide you through the claiming process. Can you please confirm your mailing address so I can send you the prize notification letter?

--------------------------

Anxious

In [22]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:ot5GP]

--------------------------------------------------------------------------------
suspect (to anxious):

Hello, is this Mrs. Johnson? Congratulations, you've been selected as a winner of our national sweepstakes! You've won a cash prize of $10,000. Can you confirm your full name and address so I can process your reward?

--------------------------------------------------------------------------------
anxious (to suspect):

Oh no, I don't know if I should be giving out that information. Is this for real? How did you get my number? I don't want to get scammed or anything.

--------------------------------------------------------------------------------
suspect (to anxious):

Mrs. Johnson, I completely understand your concerns. Let me assure you, this is a legitimate call from the National Prize Commission. We've been trying to reach you for weeks, and your number was randomly selected from a pool of entrants. I can provide you with a conf

Aggressive

In [23]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller’s attempts to extract information.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:Uphul]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hello! Congratulations, Karen! You've been selected as a winner in our recent national sweepstakes. You've won a cash prize of $10,000! Can I have you confirm your address so we can process the paperwork and get your reward to you?

--------------------------------------------------------------------------------
aggressive (to suspect):

What sweepstakes? I don't remember entering anything. How did you get my number?

--------------------------------------------------------------------------------
suspect (to aggressive):

Don't worry about that, Karen! You must have entered online or at a local retailer. We've got all the records here. Your number was selected randomly from our database. This is a legitimate contest, and you're one of the lucky winners! I just need to verify some information to process your prize. Can you plea

In [24]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:elbm5]

--------------------------------------------------------------------------------
suspect (to greedy):

Hello! My name is Karen Thompson and I'm calling from the Consumer Protection Agency. How are you today, Emily?

--------------------------------------------------------------------------------
greedy (to suspect):

I'm doing well, Karen. Thanks for asking. What can I do for the Consumer Protection Agency today? Is this about a possible refund or settlement I'm eligible for?

--------------------------------------------------------------------------------
suspect (to greedy):

You're absolutely on the right track, Emily! We've been trying to reach you about a class-action lawsuit settlement. You're eligible for a significant cash award of $9,500. It's a one-time payment for a product you purchased a few years back. Can you confirm your mailing address so I can proceed with the claim?

---------------------------------------------------

In [25]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [26]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [27]:
combinded_df.to_csv('./data/llama3_agent_reward_scam_200.csv', index=False)